In [ ]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-core==0.2.9 --progress-bar off
!pip -qqq install langchain-community==0.2.5 --progress-bar off
!pip -qqq install 'crewai[tools]'==0.32.0 --progress-bar off
!pip -qqq install langchain-groq==0.1.5 --progress-bar off
!pip -qqq install langchain-experimental==0.0.61 --progress-bar off

In [2]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task, LLM
from crewai.tools import tool
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from dotenv import  load_dotenv

load_dotenv()



True

In [3]:
df = pd.read_excel("./Files/Student Schedule Dataset with Dimension Description.xlsx", sheet_name="Sheet1")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

df.head()

,student_id,student_name,advisor_name,course_code,course_name,term,instructor_name,days,time,building,room_number,credits
0,SID01,Student 01,Advisor 2,CSC101,Intro to CS,Fall 2024,Professor 1,MWF,08:00:00,Building 1,100,3
1,SID01,Student 01,Advisor 2,CSC102,Data Structures,Fall 2024,Professor 2,TR,10:00:00,Building 2,101,4
2,SID01,Student 01,Advisor 2,CSC103,Algorithms,Fall 2024,Professor 3,MWF,13:00:00,Building 3,102,3
3,SID02,Student 02,Advisor 3,CSC104,Databases,Fall 2024,Professor 4,TR,15:00:00,Building 1,103,4
4,SID02,Student 02,Advisor 3,CSC105,Operating Systems,Fall 2024,Professor 5,MWF,08:00:00,Building 2,104,3


In [4]:
connection = sqlite3.connect("student_details.db")
df.to_sql(name="students", if_exists="replace",con=connection)

50

## LLM (Llama 3)

In [5]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str


def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()


class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

In [6]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)

my_llm = LLM(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama3-70b-8192",
)



In [7]:
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("human", human)])


my_llm=LLM(model="ollama/llama3.2", base_url="http://localhost:11434")
response = my_llm.call(
    "Which company has better models OpenAI or Anthropic? Respond with just the company name."

)

print(response)

Anthropic.


## Tools

In [8]:
db = SQLDatabase.from_uri("sqlite:///student_details.db")

In [9]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

In [10]:
list_tables.run()

Using Tool: list_tables


'students'

In [11]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

In [12]:
print(tables_schema.run("students"))

Using Tool: tables_schema

CREATE TABLE students (
	"index" INTEGER, 
	student_id TEXT, 
	student_name TEXT, 
	advisor_name TEXT, 
	course_code TEXT, 
	course_name TEXT, 
	term TEXT, 
	instructor_name TEXT, 
	days TEXT, 
	time TIME, 
	building TEXT, 
	room_number INTEGER, 
	credits INTEGER
)

/*
3 rows from students table:
index	student_id	student_name	advisor_name	course_code	course_name	term	instructor_name	days	time	building	room_number	credits
0	SID01	Student 01	Advisor 2	CSC101	Intro to CS	Fall 2024	Professor 1	MWF	08:00:00	Building 1	100	3
1	SID01	Student 01	Advisor 2	CSC102	Data Structures	Fall 2024	Professor 2	TR	10:00:00	Building 2	101	4
2	SID01	Student 01	Advisor 2	CSC103	Algorithms	Fall 2024	Professor 3	MWF	13:00:00	Building 3	102	3
*/


In [13]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

In [14]:
execute_sql.run("SELECT * FROM students ")

Using Tool: execute_sql


C:\Users\kaasa\AppData\Local\Temp\ipykernel_30224\2967140761.py:4: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  return QuerySQLDataBaseTool(db=db).invoke(sql_query)


"[(0, 'SID01', 'Student 01', 'Advisor 2', 'CSC101', 'Intro to CS', 'Fall 2024', 'Professor 1', 'MWF', '08:00:00.000000', 'Building 1', 100, 3), (1, 'SID01', 'Student 01', 'Advisor 2', 'CSC102', 'Data Structures', 'Fall 2024', 'Professor 2', 'TR', '10:00:00.000000', 'Building 2', 101, 4), (2, 'SID01', 'Student 01', 'Advisor 2', 'CSC103', 'Algorithms', 'Fall 2024', 'Professor 3', 'MWF', '13:00:00.000000', 'Building 3', 102, 3), (3, 'SID02', 'Student 02', 'Advisor 3', 'CSC104', 'Databases', 'Fall 2024', 'Professor 4', 'TR', '15:00:00.000000', 'Building 1', 103, 4), (4, 'SID02', 'Student 02', 'Advisor 3', 'CSC105', 'Operating Systems', 'Fall 2024', 'Professor 5', 'MWF', '08:00:00.000000', 'Building 2', 104, 3), (5, 'SID02', 'Student 02', 'Advisor 3', 'CSC106', 'AI Fundamentals', 'Fall 2024', 'Professor 6', 'TR', '10:00:00.000000', 'Building 3', 105, 4), (6, 'SID03', 'Student 03', 'Advisor 4', 'CSC107', 'Software Engineering', 'Fall 2024', 'Professor 7', 'MWF', '13:00:00.000000', 'Building 

In [15]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

In [16]:
check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'SELECT * FROM salaries WHERE salary > 10000 LIMIT 5'

## Agents

In [17]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=my_llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [18]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=my_llm,
    allow_delegation=False,
)

In [19]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=my_llm,
    allow_delegation=False,
)

## Tasks

In [20]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [21]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [22]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

## Crew

In [23]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

In [24]:
inputs = {
    "query": "What are the courses of Student 01?"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8b8dcad1-0ec3-46a2-be10-450c6b542a98                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Senior Database Developer

## Task: Extract data that is required for the query What are the courses of Student 01?.

Output()

: 

In [1]:
print(result)

NameError: name 'result' is not defined

In [ ]:
inputs = {
    "query": "How is the `Machine Learning Engineer` salary in USD is affected by remote positions"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
print(result)

## References

- [DS Salaries Dataset](https://huggingface.co/datasets/Einstellung/demo-salaries)